# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [2]:
# your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [8]:
col = ['target', 'id', 'date', 'flag', 'user', 'text']
tweets = pd.read_csv('../training.1600000.processed.noemoticon.csv', names = col, encoding = 'latin-1')


In [9]:
tweets2 = tweets.iloc[:20000,]

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [10]:
# your code here
tweets2.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [22]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import re


def clean_up(s):
    clean = re.sub(r'http\S+', ' ', s).replace("'", " ").lower()
    clean = re.sub(r"[^a-zA-Z]", " ", s)

    return clean

def tokenize(s):  
    token = nltk.word_tokenize(s)
    
    return token

def stem_and_lemmatize(l):
    ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stem = [lemmatizer.lemmatize(ps.stem(word)) for word in l]
    return stem

def remove_stopwords(l):
    stop_words = stopwords.words('english')
    w = [x for x in l if x not in stop_words]
    return w

In [23]:
tweets2["text_processed"] = tweets2["text"].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)


<ipython-input-23-c588c5ad8630>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets2["text_processed"] = tweets2["text"].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)


In [24]:
tweets2

,target,id,date,flag,user,text,text_processed
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","[switchfoot, http, twitpic, com, zl, awww, bum..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"[upset, updat, hi, facebook, text, might, cri,..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"[kenichan, I, dive, mani, time, ball, manag, s..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[whole, bodi, feel, itchi, like, fire]"
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","[nationwideclass, behav, mad, whi, becaus, I, ..."
...,...,...,...,...,...,...,...
19995,0,1556975331,Sun Apr 19 01:19:14 PDT 2009,NO_QUERY,TOMurdockPapers,"Not much time off this weekend, work trip to M...","[much, time, thi, weekend, work, trip, malm, f..."
19996,0,1556976068,Sun Apr 19 01:19:30 PDT 2009,NO_QUERY,nikibennn,One more day of holidays,"[one, day, holiday]"
19997,0,1556976167,Sun Apr 19 01:19:32 PDT 2009,NO_QUERY,eifflesummer,feeling so down right now .. i hate you DAMN H...,"[feel, right, hate, damn, humprey]"
19998,0,1556976222,Sun Apr 19 01:19:34 PDT 2009,NO_QUERY,lomobabes,"geez,i hv to READ the whole book of personalit...","[geez, hv, read, whole, book, person, type, be..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [51]:
from nltk.probability import ConditionalFreqDist
from nltk.tokenize import word_tokenize

# your code here
cfdist = nltk.FreqDist()

for i in tweets2["text_processed"]:
    for word in i:
        cfdist[word] += 1
    
    

In [57]:
top5000 = list(cfdist.keys())[:5000]
top5000

['switchfoot',
 'http',
 'twitpic',
 'com',
 'zl',
 'awww',
 'bummer',
 'shoulda',
 'got',
 'david',
 'carr',
 'third',
 'day',
 'D',
 'upset',
 'updat',
 'hi',
 'facebook',
 'text',
 'might',
 'cri',
 'result',
 'school',
 'today',
 'also',
 'blah',
 'kenichan',
 'I',
 'dive',
 'mani',
 'time',
 'ball',
 'manag',
 'save',
 'rest',
 'go',
 'bound',
 'whole',
 'bodi',
 'feel',
 'itchi',
 'like',
 'fire',
 'nationwideclass',
 'behav',
 'mad',
 'whi',
 'becaus',
 'see',
 'kwesidei',
 'crew',
 'need',
 'hug',
 'loltrish',
 'hey',
 'long',
 'ye',
 'rain',
 'bit',
 'onli',
 'lol',
 'fine',
 'thank',
 'tatiana',
 'K',
 'nope',
 'twittera',
 'que',
 'muera',
 'spring',
 'break',
 'plain',
 'citi',
 'snow',
 'pierc',
 'ear',
 'caregiv',
 'bear',
 'watch',
 'thought',
 'UA',
 'loss',
 'wa',
 'embarrass',
 'octolinz',
 'It',
 'count',
 'idk',
 'either',
 'never',
 'talk',
 'anymor',
 'smarrison',
 'would',
 'first',
 'gun',
 'realli',
 'though',
 'zac',
 'snyder',
 'doucheclown',
 'iamjazzyfizzl'

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [66]:
# your code here
def find_features(document):
    words = set(document)
    features = {}
    for w in top5000:
        features[w] = (w in words)
    return features


In [67]:
documents = list(zip(tweets2['text_processed'], np.where(tweets2['target'] == 4, True, False)))

featuresets = [(find_features(rev), category) for (rev, category) in documents]
featuresets[0]

({'switchfoot': True,
  'http': True,
  'twitpic': True,
  'com': True,
  'zl': True,
  'awww': True,
  'bummer': True,
  'shoulda': True,
  'got': True,
  'david': True,
  'carr': True,
  'third': True,
  'day': True,
  'D': True,
  'upset': False,
  'updat': False,
  'hi': False,
  'facebook': False,
  'text': False,
  'might': False,
  'cri': False,
  'result': False,
  'school': False,
  'today': False,
  'also': False,
  'blah': False,
  'kenichan': False,
  'I': False,
  'dive': False,
  'mani': False,
  'time': False,
  'ball': False,
  'manag': False,
  'save': False,
  'rest': False,
  'go': False,
  'bound': False,
  'whole': False,
  'bodi': False,
  'feel': False,
  'itchi': False,
  'like': False,
  'fire': False,
  'nationwideclass': False,
  'behav': False,
  'mad': False,
  'whi': False,
  'becaus': False,
  'see': False,
  'kwesidei': False,
  'crew': False,
  'need': False,
  'hug': False,
  'loltrish': False,
  'hey': False,
  'long': False,
  'ye': False,
  'rain': 

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [71]:
# your code here
train_set, test_set = featuresets[:10000], featuresets[10000:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [73]:
# your code here
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(15)

1.0
Most Informative Features


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here